<a href="https://colab.research.google.com/github/arezoosh/PhD-Course1/blob/master/tweet-preprocessor-portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Perform an LDA analysis of the #OKBoomer dataset

#Filter the corpus using tweet-preprocessor - try to figure out how to use it using it's documentation Clean up further with SpaCy (keep only ADV, ADJ, NOUN) Use Gensim to build a Dictionary (Filter extremes) and Corpus Use Gensim to run LDA Identify 10 topics Plot topic-counts by day

In [4]:
!pip install tweet-preprocessor

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7947 sha256=cc745d278bfa84a3f988aa5886037f0e37b3e620789add34e67bdff25ff26266
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor


In [0]:
import pandas as pd
import spacy
import preprocessor as p
from nltk.tokenize import TweetTokenizer

In [0]:
# download and open some boomer tweets from trump_tweet_data_archive

boomer_tweets_df = pd.read_json('https://github.com/CALDISS-AAU/sdsphd19_coursematerials/raw/master/data/tweets_boomer.zip')


In [0]:
boomer_clean = boomer_tweets_df.tweet.map(p.clean)

In [0]:

tknzr = TweetTokenizer(strip_handles=True)
boomer_tweets_df ['tokenized']= boomer_clean.map(lambda t: tknzr.tokenize(t))
boomer_tweets_df ['tokenized']= boomer_tweets_df ['tokenized'].map(lambda t: [token.strip('@') for token in t if token.isalpha()])

In [0]:
boomer_tweets_df ['tokenized']= boomer_tweets_df ['tokenized'].map(lambda t: ' '.join(t))

In [0]:
nlp = spacy.load("en")

In [0]:
tokens =[]
for tweet in nlp.pipe(boomer_tweets_df ['tokenized']):
    proj_tok = [token.lemma_.lower() for token in tweet if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] and not token.is_stop] 
    tokens.append(proj_tok)

In [19]:
boomer_tweets_df ['tokenized_sp']= tokens
boomer_tweets_df ['tokenized_sp']


0                                 [suddenly]
1                                     [role]
2                        [allot, old, white]
3                    [university, president]
4                  [conway, fragile, fossil]
                        ...                 
10013                                 [lmao]
10014                         [good, thread]
10015    [boomer, not, new, boss, old, boss]
10016                                 [room]
10017                                [legit]
Name: tokenized_sp, Length: 9722, dtype: object

In [20]:
!pip install -qq -U gensim

     |████████████████████████████████| 24.2MB 99kB/s 


In [0]:
from gensim.corpora.dictionary import Dictionary

In [0]:
dictionary = Dictionary(boomer_tweets_df['tokenized_sp'])

In [0]:
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=5, no_above=0.1, keep_n=1000)

In [0]:
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(doc) for doc in boomer_tweets_df['tokenized_sp']]

In [0]:
from gensim.models import LdaMulticore

In [0]:
# Training the model (makes some mess atm due to version clashes)

lda_model = LdaMulticore(corpus, id2word=dictionary,  num_topics=10, workers = 4, passes=10)

In [30]:
lda_model.print_topics(-1)

[(0,
  '0.019*"people" + 0.018*"boomer" + 0.011*"right" + 0.010*"generation" + 0.009*"moment" + 0.008*"perfect" + 0.008*"thank" + 0.007*"age" + 0.007*"point" + 0.006*"good"'),
 (1,
  '0.037*"way" + 0.033*"boomer" + 0.027*"meme" + 0.024*"people" + 0.014*"young" + 0.013*"new" + 0.013*"old" + 0.011*"generation" + 0.009*"congratulations" + 0.009*"purchase"'),
 (2,
  '0.032*"boomer" + 0.015*"old" + 0.009*"people" + 0.009*"time" + 0.008*"boomers" + 0.008*"generation" + 0.007*"millennial" + 0.007*"gen" + 0.006*"phone" + 0.006*"pink"'),
 (3,
  '0.046*"t" + 0.026*"don" + 0.016*"people" + 0.014*"cherry" + 0.011*"old" + 0.010*"boomer" + 0.009*"generation" + 0.009*"time" + 0.007*"young" + 0.006*"good"'),
 (4,
  '0.019*"people" + 0.016*"boomer" + 0.016*"time" + 0.011*"good" + 0.011*"twitter" + 0.011*"old" + 0.011*"guy" + 0.010*"kid" + 0.010*"word" + 0.010*"thing"'),
 (5,
  '0.024*"generation" + 0.014*"people" + 0.010*"day" + 0.008*"right" + 0.008*"age" + 0.008*"millenial" + 0.008*"insult" + 0.008*"

In [32]:
!pip install -qq pyLDAvis

     |████████████████████████████████| 1.6MB 9.0MB/s 
     |████████████████████████████████| 552kB 42.2MB/s 


In [0]:
# and import it
import pyLDAvis.gensim
%matplotlib inline
pyLDAvis.enable_notebook()

In [0]:
# Let's try to visualize
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)


In [0]:
pyLDAvis.display(lda_display)

In [0]:
# In case you run a website and want to publish it...or embed it in a blogpost...
pyLDAvis.save_html(lda_display, 'lda.html')

In [0]:
lda_model[corpus][8]

In [0]:
 # that's how you get the topic-number that's ranked highest

print(sorted([(2, 0.121567), (9, 0.8610384)], key=lambda x: -x[1]))
print(sorted([(2, 0.121567), (9, 0.8610384)], key=lambda x: -x[1])[0][0])